In [53]:
using ITensors

# Define Parameters

In [76]:
# Site number N 
N = 3;

# Spin quantum number S 
S = 1;

# Max number of sweeps
max_sweep = 80;

# Max number of bond dimensions
D_max = 100;

# Max number of excited states calculated
max_states = 4;

# Penalty weight when calculating the excited states
weight = 100;

# Define Hamiltonians

### AKLT Hamiltonian

In [77]:
function AKLT_Hamiltonian(N::Int64)

    os = OpSum() 
    for j=1:N-1 
        # S*S Terms
        # Sz Sz
        os += "Sz", j, "Sz", j+1

        # S+ S- and S- S+
        os += 1/2, "S+", j, "S-", j+1
        os += 1/2, "S-", j, "S+", j+1

        # Square terms in (S*S)^2
        # (1/3) * (Sz Sz)^2 term
        os += (1/3), "Sz", j, "Sz", j+1, "Sz", j, "Sz", j+1
        # (1/3) * (1/4) (S+ S- S+ S-) term
        os += (1/3) * (1/4), "S+", j, "S-", j+1, "S+", j, "S-", j+1
        os += (1/3) * (1/4), "S-", j, "S+", j+1, "S-", j, "S+", j+1

        # Cross terms in (S*S)^2
        # (1/3) * (1/2) Sz Sz S+ S- terms
        os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S+", j, "S-", j+1
        os += (1/3) * (1/2), "S+", j, "S-", j+1, "Sz", j, "Sz", j+1

        # (1/3) * (1/2) Sz Sz S- S+ terms
        os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S-", j, "S+", j+1
        os += (1/3) * (1/2), "S-", j, "S+", j+1, "Sz", j, "Sz", j+1

        # (1/3) * (1/4) S+ S- S- S+ terms
        os += (1/3) * (1/4), "S+", j, "S-", j+1, "S-", j, "S+", j+1
        os += (1/3) * (1/4), "S-", j, "S+", j+1, "S+", j, "S-", j+1
    end

    return os
end

AKLT_Hamiltonian (generic function with 1 method)

### Heisenberg Hamiltonian

In [78]:
function Heisenberg_Hamiltonian(N::Int64, J::Any)
    """
    This function is used to generate Hamiltonian with Opsum of the Heisenberg model
    N: the number of sites
    J: J coupling
    """
    os = OpSum()
    for n=1:N-1
        os += J, "Sz",n,"Sz",n+1
        os += J/2, "S+", n, "S-", n+1
        os += J/2, "S-", n, "S+", n+1
    end

    # periodic boundary conditions (PBCs)
    #os += J, "Sz",1,"Sz",N
    #os += J/2, "S+", 1, "S-", N
    #os += J/2, "S-", 1, "S+", N

    return os
end

Heisenberg_Hamiltonian (generic function with 1 method)

# AKLT Simulation

### DMRG Parameters

In [79]:
BondDims = [10, 20, 40, D_max];
cutoff = [1e-10];
noise = [1e-7, 1e-8, 1e-10, 0, 1e-11, 1e-10, 1e-9, 1e-11, 0];

### Site and State Initialization

In [80]:
# Define site according to the spin type 
if S == 0.5
    sites = siteinds("S=1/2",N);
 elseif S == 1
    sites = siteinds("S=1",N);
 else
    println("Other spin model currently not available :(");
 end

3-element Vector{Index{Int64}}:
 (dim=3|id=946|"S=1,Site,n=1")
 (dim=3|id=920|"S=1,Site,n=2")
 (dim=3|id=353|"S=1,Site,n=3")

In [81]:
function InitMPS(linkdim::Int, K::Int, WaveType::String, sites::Vector{Index{Int64}})
    # Create an array containing all the states we want to calculate 
    psi_arr = []
   
    if WaveType == "random"
       println("start with random states")
       for m = 1:K
           # Generate a random MPS
           psi_m = randomMPS(sites; linkdims = linkdim)
           # Add the new MPS to the array
           push!(psi_arr, psi_m)
       end
    else
       println("which initial states?!")
    end
    return psi_arr
end

InitMPS (generic function with 1 method)

In [82]:
# Create a random array of MPS using the function we just defined
psi_initials = InitMPS(10, max_states, "random", sites);

# Create a AKLT Hamiltonian MPO
os = AKLT_Hamiltonian(N);
H_AKLT = MPO(os, sites);

# Define an observer to control the energy tolerance
obs = DMRGObserver(["Sz"], sites, energy_tol=1E-17);

start with random states


### Ground State Calculation

In [83]:
println("Start Calculating Groud State!")

# Use DMRG to calculate the ground state
temp_energy, temp_psi = dmrg(H_AKLT, psi_initials[1], nsweeps=max_sweep, maxdim=BondDims, mindim=20, cutoff=cutoff, noise=noise, eigsolve_krylovdim=9, observer = obs);

# Calculate the norm of optimized ground state MPS
norm = inner(temp_psi, temp_psi);

# Calculate the normalized ground state energy
energy = temp_energy/norm;
println("The ground state energy is: ", energy)

Start Calculating Groud State!
After sweep 1 energy=-1.3333333333333333  maxlinkdim=9 maxerr=0.00E+00 time=0.040
After sweep 2 energy=-1.3333333333333335  maxlinkdim=9 maxerr=0.00E+00 time=0.008
After sweep 3 energy=-1.3333333333333333  maxlinkdim=9 maxerr=0.00E+00 time=0.008
After sweep 4 energy=-1.3333333333333333  maxlinkdim=3 maxerr=8.19E-17 time=0.031
Energy difference less than 1.0e-17, stopping DMRG
The ground state energy is: -1.333333333333333


### Calculate Excited States

In [84]:
# Store the result in the energy array
energy_array = [energy];
psi_array = [temp_psi];

for m = 2:max_states
  println("Start Calculating the $(m-1)^th Excited State!")

  # Caculate the m^th excited state
  temp_energy, temp_psi = dmrg(H_AKLT, psi_array[1:m-1], psi_initials[m]; nsweeps=max_sweep, maxdim=BondDims, mindim=20, cutoff=cutoff, noise=noise, weight=weight, eigsolve_krylovdim=9, observer=obs);
  norm = inner(temp_psi, temp_psi);
  energy = temp_energy/norm;
  println("The $(m-1)^th excited state energy is", energy)

  # Store the resulting state and energy
  push!(psi_array, temp_psi);
  push!(energy_array, energy);
end

Start Calculating the 1^th Excited State!
After sweep 1 energy=-1.3333333333333333  maxlinkdim=9 maxerr=0.00E+00 time=0.015
Energy difference less than 1.0e-17, stopping DMRG
The 1^th excited state energy is-1.333333333333333
Start Calculating the 2^th Excited State!
After sweep 1 energy=-1.3333333333333335  maxlinkdim=9 maxerr=0.00E+00 time=0.006
After sweep 2 energy=-1.3333333333333333  maxlinkdim=9 maxerr=0.00E+00 time=0.004
After sweep 3 energy=-1.3333333333333335  maxlinkdim=9 maxerr=0.00E+00 time=0.011
After sweep 4 energy=-1.3333333333333333  maxlinkdim=3 maxerr=3.83E-17 time=0.014
After sweep 5 energy=-1.3333333333333335  maxlinkdim=9 maxerr=0.00E+00 time=0.012
After sweep 6 energy=-1.3333333333333333  maxlinkdim=9 maxerr=0.00E+00 time=0.015
After sweep 7 energy=-1.3333333333333335  maxlinkdim=9 maxerr=0.00E+00 time=0.027
After sweep 8 energy=-1.333333333333334  maxlinkdim=9 maxerr=0.00E+00 time=0.022
After sweep 9 energy=-1.333333333333333  maxlinkdim=3 maxerr=7.94E-17 time=0.

In [85]:
println(energy_array)

[-1.333333333333333, -1.333333333333333, -1.3333333333333328, -1.3333333333333326]
